In [1]:
import vessel_scoring.models
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt

models = vessel_scoring.models.load_models()

def messages_from_bq_dump(path):
    with open(path) as f:
        for row in json.load(f):
            row = row
            row = {key: float(value) for (key, value) in row.items()
                if key in ('timestamp', 'course', 'speed', 'lat', 'lon')
                  and value is not None}
            row['timestamp'] = datetime.datetime.fromtimestamp(int(row['timestamp']))
            yield row

messages = models['Logistic'].predict_messages(
    messages_from_bq_dump("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\Spire_test.json"))

In [13]:
print(list(messages))

[]


In [12]:
f = open("C:\\Users\\MarkR\\Documents\\UTwente\\GFW model\\Git clone scoring\\vessel-scoring\\datasets\\Spire_test.json")

all_rows = []
counter = 0
for row in json.load(f):
    counter += 1
    row = {key: float(value) for (key, value) in row.items()
                if key in ('timestamp', 'course', 'speed', 'lat', 'lon')
                  and value is not None}
    row['timestamp'] = datetime.datetime.fromtimestamp(int(row['timestamp']))
    # print(counter, row)
    all_rows.append(row)

print(all_rows[0])


{'timestamp': datetime.datetime(2022, 1, 6, 1, 1, 45), 'course': 144.0, 'speed': 0.2, 'lat': 1.5738166667, 'lon': 104.81515}


In [28]:
import vessel_scoring.add_measures
from vessel_scoring import logistic_model
import six

msg = list(vessel_scoring.add_measures.AddMeasures(all_rows))[0]

msg['measure_new_score'] = float(self.predict_proba({
                            key: [value]
                            for key, value in six.iteritems(msg)
                            })[0][1])


TypeError: predict_proba() missing 1 required positional argument: 'X'

In [3]:
messages = list(messages)

m = np.zeros(len(messages), dtype=[
        (name, 'float') for name in (
            'lat','lon','measure_new_score','timestamp')])

for idx, msg in enumerate(messages):
    for key in m.dtype.names:
        if key not in msg: continue
        val = msg[key]
        if key == 'timestamp':
            val = (val - datetime.datetime(1970, 1, 1)).total_seconds()
        m[idx][key] = val

plt.plot(m['timestamp'], m['measure_new_score'], color='red')
plt.show()

NotFittedError: This LogisticModel instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.